In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=6c39001eb820f52e98a83c62500d91bfa94448b24bb9c7f5b740463c0036dade
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-i8wvmd9f/kobert-tokenizer_9f001e69ccae4d2e80ca12b6dc11f4a8
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-i8wvmd9f/kobert-tokenizer_9f001e69ccae4d2e80ca12b6dc11f4a8
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=f5da360c0a98bb537f1abfa707ec783bef09290f751ad460900c665012586662
  Stored in directory: /tmp/pip-ephem-wheel-cache-sigfe_cv/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from gluonnlp.vocab import BERTVocab
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [5]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
device = torch.device("cuda:0")

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
# vocab 객체 생성
vocab = BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [8]:
ls ./drive/MyDrive

 Final/                   kobert_test.ipynb                                   python3.9.ipynb
'kobert_test (1).ipynb'   naver_review_classifications_pytorch_kobert.ipynb   Untitled0.ipynb


In [10]:
ls ./drive/MyDrive/Final/data

p1_EC.csv


In [11]:
import pandas as pd

In [15]:
df_EC = pd.read_csv('./drive/MyDrive/Final/data/p1_EC.csv')

In [16]:
df_EC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42063 entries, 0 to 42062
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Unnamed: 0                         42063 non-null  int64 
 1   sourceDataInfo_newsID              42063 non-null  object
 2   sourceDataInfo_newsCategory        42063 non-null  object
 3   sourceDataInfo_newsSubcategory     41853 non-null  object
 4   sourceDataInfo_newsTitle           42063 non-null  object
 5   sourceDataInfo_newsSubTitle        24578 non-null  object
 6   sourceDataInfo_newsContent         42063 non-null  object
 7   sourceDataInfo_partNum             42063 non-null  object
 8   sourceDataInfo_useType             42063 non-null  int64 
 9   sourceDataInfo_processType         42063 non-null  object
 10  sourceDataInfo_processPattern      42063 non-null  int64 
 11  sourceDataInfo_processLevel        42063 non-null  object
 12  sour

In [17]:
df_EC.head()

,Unnamed: 0,sourceDataInfo_newsID,sourceDataInfo_newsCategory,sourceDataInfo_newsSubcategory,sourceDataInfo_newsTitle,sourceDataInfo_newsSubTitle,sourceDataInfo_newsContent,sourceDataInfo_partNum,sourceDataInfo_useType,sourceDataInfo_processType,sourceDataInfo_processPattern,sourceDataInfo_processLevel,sourceDataInfo_sentenceCount,sourceDataInfo_sentenceInfo,labeledDataInfo_newTitle,labeledDataInfo_clickbaitClass,labeledDataInfo_referSentenceInfo
0,0,EC_M02_000001,경제,일반,"[오늘의 SR이슈] 윤리 문제 불거진 종근당 회장 조사 마치고 귀가…\""사죄한다\""",NaN,"\""사죄한다.\"" 운전기사들에게 허구한 날 폭언을 퍼부어 윤리 문제가 불거진 이장한...",P1,0,A,99,하,9,"[{'sentenceNo': 1, 'sentenceContent': '\\""사죄한다...",'친환경 금융 목표'…'탄소중립' 실천 나선 금융업계,0,"[{'sentenceNo': 1, 'referSentenceyn': 'N'}, {'..."
1,1,EC_M02_000005,경제,은행,"최종구 금융위원장, 임팩트금융 정착 위해 정부 차원 기금 조성",NaN,"최종구 금융위원회 위원장이 \""'임팩트금융(사회적금융)'의 안착 차원에서 정부가 직...",P1,0,A,99,하,13,"[{'sentenceNo': 1, 'sentenceContent': '최종구 금융위...",공동주택 하자분쟁 조정 본격화,0,"[{'sentenceNo': 1, 'referSentenceyn': 'N'}, {'..."
2,2,EC_M02_000006,경제,은행,한국기업지배구조원 733개 기업 ESG 등급 부여,NaN,"한국기업지배구조원이 8월 31일 국내 상장회사의 환경경영(E), 사회책임경영(S),...",P1,0,A,99,하,13,"[{'sentenceNo': 1, 'sentenceContent': '한국기업지배구...","LG생활건강, 사상 최대 실적에도 주가 반등은 '글쎄'",0,"[{'sentenceNo': 1, 'referSentenceyn': 'N'}, {'..."
3,3,EC_M02_000007,경제,산업,가을 맞아 더 활발한 기업 프로보노 활동,NaN,요즘 프로보노 형 사회 공헌 활동을 펴는 회사가 늘고 있다.\n회사를 경영하거나 기...,P1,0,A,99,하,13,"[{'sentenceNo': 1, 'sentenceContent': '요즘 프로보노...","로봇청소기 강자 코오롱글로벌, '드리미 W10' 국내 론칭",0,"[{'sentenceNo': 1, 'referSentenceyn': 'N'}, {'..."
4,4,EC_M02_000008,경제,일반,2017 다우존스 지속가능경영지수 어떤 기업이 편입되었나?,NaN,"런던 현지 시각으로 7일, 스탠다드앤푸어스 다우존스 인디시즈와 로베코샘이 다우존스지...",P1,0,A,99,하,9,"[{'sentenceNo': 1, 'sentenceContent': '런던 현지 시...","한숨만 쉬는 유통업계, 매출 얼마나 줄었나 봤더니…",0,"[{'sentenceNo': 1, 'referSentenceyn': 'N'}, {'..."


In [18]:
from sklearn.model_selection import train_test_split

In [24]:
df_test = df_EC.iloc[:, [4,6,15]]

In [68]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [26]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [66]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx_1, sent_idx_2, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx_1]]) for i in dataset]
        self.sentences = [transform([i[sent_idx_2]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [60]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42063 entries, 0 to 42062
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   sourceDataInfo_newsTitle        42063 non-null  object
 1   sourceDataInfo_newsContent      42063 non-null  object
 2   labeledDataInfo_clickbaitClass  42063 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 986.0+ KB


In [61]:
data_list = []

for title, content, label in zip(df_test['sourceDataInfo_newsTitle'], df_test['sourceDataInfo_newsContent'],
                                 df_test['labeledDataInfo_clickbaitClass']):
  data = []
  data.append(title)
  data.append(content)
  data.append(str(label))

  data_list.append(data)

In [ ]:
data_list

In [59]:
for i in train:
  print(i)

sourceDataInfo_newsTitle
sourceDataInfo_newsContent
labeledDataInfo_clickbaitClass


In [42]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [63]:
train, test = train_test_split(data_list, test_size = 0.2, random_state = 42)

In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33650 entries, 29847 to 15795
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   sourceDataInfo_newsTitle        33650 non-null  object
 1   sourceDataInfo_newsContent      33650 non-null  object
 2   labeledDataInfo_clickbaitClass  33650 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [67]:
data_train = BERTDataset(train, 0, 1, 2, tokenizer, vocab, max_len, True, False)

data_test = BERTDataset(test, 0, 1, 2, tokenizer, vocab, max_len, True, False)

In [47]:
print(train['labeledDataInfo_clickbaitClass'].unique())

[1 0]


In [69]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [70]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [71]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [72]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [73]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [74]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [75]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [77]:
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

In [78]:
train_dataloader

In [81]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-81-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/526 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 1.550392508506775 train acc 0.40625
epoch 1 batch id 201 loss 0.6327860355377197 train acc 0.6038557213930348
epoch 1 batch id 401 loss 0.4092591106891632 train acc 0.6843827930174564
epoch 1 train acc 0.7095484790874526


<ipython-input-81-c43874aa7363>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 9 test acc 0.8547544409613376


  0%|          | 0/526 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0027602750342339277 train acc 1.0
epoch 10 batch id 201 loss 0.1535998433828354 train acc 0.9955690298507462
epoch 10 batch id 401 loss 0.10343129187822342 train acc 0.9956359102244389
epoch 10 train acc 0.9954550855513308


  0%|          | 0/132 [00:00<?, ?it/s]

epoch 10 test acc 0.8548728121734588
